In [6]:
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
options(repr.matrix.max.rows = 6)
set.seed(100)

# Read Me:


## comment from Trevor:
### 1. tansfrom the predictors showing huge range into log scale before any other recipe steps. Check if there is any built in log special step in recipe? 

  
### 2. Do you need to upsample? it is not super unbalanced 
### - use cross-validation to compare K = 1, 2, 3, ... 
### - then compare accuracy on the data without upsample and with upsample, see if upsample really improve the result. 

### Rock Physical Properties and Rock Type 
**DSCI 100 Project Final Report
**Group 4: Naijide Anwaer, Maggie Ruan, Yun Xing  
Date: December 05, 2021**


### Introduction 
The purpose of this project proposal is to provide an overview of the rock petrophysical data process project. This proposal introduces the rock properties database, illustrates the exploratory data analysis, explains the methodology, and discusses the expected outcome of this project.  
 
The Canadian Rock Physical Property Database (CRPPD) includes rock petrophysical measurements on over 20,000 observations collected during the last 40 years. The rock measurements available in this database are density, porosity, resistivity etc.. Geographically, observations are categorized into different sample types including outcrop, subcrop, hand sample, and borehole sample. Geologically, observations are categorized into different rock types including igneous, sedimentary, and metamorphic, the three fundamental rock types.  

The group intends to study the relationship between the rock petrophysical measurements and the three fundamental rock types, and to answer a classification problem: given the physical properties data, whether the rock will be igneous, sedimentary, or metamorphic.   

The classifier model will be useful in the situation where the physical property data are available while the rock type is unclassified. An example of this will be in diamond drillholes (sample type: Borehole), where not all the rock core in the borehole are brought up to the surface for visual examination. As a result, the group decided to perform the data analysis with the focus on data categorized under sample type borehole.



### Preliminary Analysis
The preliminary data are obtained as a zip file from the Government of Canada, Natural Resources Canada website. Upon unzipping the file and bringing the original excel file into Jupiter notebook for R processing, the group noticed that R encountered a runtime error.  
 
Indicated by read_* documentation, the column type will be imputed from the first 1000 rows on the input file. As the top 1000 data in this excel mostly contains NULL values, R automatically assigned “col_logical” as the column type for most of the columns. However, this column type conflicts with the majority of the other observations, which contain numerical values.    
 
To solve this issue, the group decides to sort the observations in excel before importing into Jupiter. We relocated the columns with no missing data to the top of the file, and R successfully created columns with the proper type “col_double” and no error was thrown. The following code demonstrates the steps of data import:


In [7]:
url <- "https://raw.githubusercontent.com/yxing6/DSCI_100_Project_Group4/main/data/of_8460_database.csv"
# download.file(url, destfile = "rock_physical_properties.csv")
# rock_data_raw <- read_csv("rock_physical_properties.csv")

rock_data_raw <- read_csv(url)

# glimpse(rock_data_all)

Parsed with column specification:
cols(
  .default = col_character(),
  `Date Mapped (yyyy/mm/dd) *` = col_double(),
  `Lat Deg` = col_double(),
  `Long Deg` = col_double(),
  `UTM zone` = col_double(),
  Easting = col_double(),
  Northing = col_double(),
  `Method Precision (m)*` = col_double(),
  `Year *` = col_double(),
  `Lab Date` = col_double(),
  `GRAIN DEN Sample Value [g/cm3]` = col_double(),
  `DRY BULK DEN Sample Value [g/cm3]` = col_double(),
  `SAT BULK DEN Sample Value [g/cm3]` = col_double(),
  `POR Sample Value [%]` = col_double(),
  `MS Sample Value [SI A/m / A/m]` = col_double(),
  `NRM Sample Value [A/m]` = col_double(),
  `KN50 Sample Value` = col_double(),
  `Local Field [uT]` = col_double(),
  KN = col_double(),
  `RES Sample Value [Ohm.m]` = col_double(),
  `CHG Sample Value [ms]` = col_double()
)

See spec(...) for full column specifications.



  
The following codes demonstrate the steps of data cleaning and wrangling into a tidy format:  
  

In [9]:
borehole_data_raw <- rock_data_raw %>% 
    select("Location Type *", "MIRA Master Litho 1", "GRAIN DEN Sample Value [g/cm3]",
           "POR Sample Value [%]","MS Sample Value [SI A/m / A/m]",
           "NRM Sample Value [A/m]","RES Sample Value [Ohm.m]","CHG Sample Value [ms]") %>%
    rename("sample_type" = "Location Type *",
          "lithology" = "MIRA Master Litho 1",
           "density" = "GRAIN DEN Sample Value [g/cm3]",
           "porosity" = "POR Sample Value [%]",
           "MS" = "MS Sample Value [SI A/m / A/m]",
           "NRM" = "NRM Sample Value [A/m]",
           "RES" = "RES Sample Value [Ohm.m]",
           "chargeability" = "CHG Sample Value [ms]") %>%
    filter(lithology != "Other", sample_type == "Borehole") %>%
    mutate(sample_type = as_factor(sample_type),
          lithology = as_factor(lithology))
    

borehole_data_raw

# column units:
# density[g/cm3]
# porosity[%]
# MS[m3/kg]
# NRM[A/m]
# RES[Ohm.m]
# chargeability[ms]

#!is.na(col)

# currently omiting all NA data 
# if later on decide on specific predictors, only omit NA on those variables 
# e.g. na.omit(borehole_data[1:3])


borehole_data_unscaled <- na.omit(borehole_data_raw) %>%
    select(-sample_type)

borehole_data_unscaled

sample_type,lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Borehole,Igneous,2.931,0.41,0.00118,0.029,7960,46.4
Borehole,Igneous,2.770,0.28,0.07900,4.810,135000,19.5
Borehole,Igneous,2.754,0.28,0.01260,0.352,51800,1.6
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Borehole,Metamorphic,2.665,0.08,NA,NA,91800,3.868
Borehole,Metamorphic,2.641,0.09,NA,NA,67800,1.388
Borehole,Metamorphic,2.811,0.36,0.0739,5.72,437000,12.850


lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Igneous,2.931,0.41,0.00118,0.029,7960,46.4
Igneous,2.770,0.28,0.07900,4.810,135000,19.5
Igneous,2.754,0.28,0.01260,0.352,51800,1.6
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Metamorphic,2.660,0.34,-7.08e-06,0.000162,18600,1.015
Metamorphic,2.791,0.16,1.68e-04,0.000401,27100,8.026
Metamorphic,2.811,0.36,7.39e-02,5.720000,437000,12.850



We then split the data into training and testing sets.  
  
Using only the training set, we built a table that consists the mean values of each predictors for each category.  

### NEW from YUN 
### We furthur computed the (range difference over mean) to see if we need to perform log transformation for any of the predictors. 
### table included below:

  

In [12]:
borehole_data_split <- initial_split(borehole_data_unscaled, prop = 0.75, strata = lithology)
data_train <- training(borehole_data_split)
data_test <- testing(borehole_data_split)


# options(repr.plot.width = 15, repr.plot.height = 15)
# train_pairs <- data_train %>%
#     select("density":"chargeability") %>%
#     ggpairs() + 
#     theme(text = element_text(size = 20))
# train_pairs

data_summary <- data_train %>%
    group_by(lithology) %>%
    summarize(n = n(), 
              across(density:chargeability, mean, na.rm = TRUE)) %>%
    rename(
        "lithology" = "lithology",
        "n" = "n",
        "mean_Density[g/cm3]" = "density",
        "mean_Porosity[%]" = "porosity",
        "mean_MS[m3/kg]" = "MS",
        "mean_NRM[A/m]" = "NRM",
        "mean_RES[Ohm.m]" = "RES",
        "mean_Chargeability[ms]" = "chargeability")

data_summary


# unbalanced data 


data_min <- data_train %>%
    group_by(lithology) %>%
    summarize(across(density:chargeability, min, na.rm = TRUE)) # %>%
#     rename(
#         "min_Density[g/cm3]" = "density",
#         "min_Porosity[%]" = "porosity",
#         "min_MS[m3/kg]" = "MS",
#         "min_NRM[A/m]" = "NRM",
#         "min_RES[Ohm.m]" = "RES",
#         "min_Chargeability[ms]" = "chargeability")
data_min


data_max <- data_train %>%
    group_by(lithology) %>%
    summarize(across(density:chargeability, max, na.rm = TRUE))  # %>%
#     rename(
#         "max_Density[g/cm3]" = "density",
#         "max_Porosity[%]" = "porosity",
#         "max_MS[m3/kg]" = "MS",
#         "max_NRM[A/m]" = "NRM",
#         "max_RES[Ohm.m]" = "RES",
#         "max_Chargeability[ms]" = "chargeability")
data_max

value_differences <- cbind(data_summary[1], round((data_max[2:7] - data_min[2:7]) / data_summary[3:8], 1))
value_differences

# levels(data_train$lithology)


`summarise()` ungrouping output (override with `.groups` argument)



lithology,n,mean_Density[g/cm3],mean_Porosity[%],mean_MS[m3/kg],mean_NRM[A/m],mean_RES[Ohm.m],mean_Chargeability[ms]
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Igneous,213,2.789535,2.2687793,0.0219035573,1.40189862,12702.906,11.134845
Metamorphic,559,2.771118,0.4513059,0.0015630911,0.51801182,10096.520,5.917313
Sedimentary,141,2.657340,6.2145390,0.0001261087,0.01746351,3670.489,3.892787


`summarise()` ungrouping output (override with `.groups` argument)



lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Igneous,2.448,0.01,-5.60e-06,3.95e-05,11.0,0.001
Metamorphic,2.599,0.02,-5.64e-06,2.36e-05,27.5,0.068
Sedimentary,2.586,0.24,-1.17e-05,9.20e-05,137.0,0.272


`summarise()` ungrouping output (override with `.groups` argument)



lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Igneous,3.734,21.75,0.53900,29.80,198000,91.04
Metamorphic,3.505,9.39,0.10300,91.70,628000,179.80
Sedimentary,2.935,16.28,0.00888,1.17,65000,43.10


lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Igneous,0.5,9.6,24.6,21.3,15.6,8.2
Metamorphic,0.3,20.8,65.9,177.0,62.2,30.4
Sedimentary,0.1,2.6,70.5,67.0,17.7,11.0


## New from Yun: 
## Except density, all other physical properties have a range of bigger than 10. Thus, we will log transformation for these predictors in the model recipe 

In [25]:
# balance and upsample

# library(themis)

negative_MS <- data_train %>%
    filter(MS < 0)
negative_MS

new_training <- setdiff(data_train, negative_MS)

test_recipe <- recipe(lithology ~ ., data = new_training) %>%
#          step_upsample(lithology, over_ratio = 1, skip = FALSE) %>%
         step_log(porosity, MS, NRM, RES, chargeability) %>%
         step_scale(all_predictors()) %>%
         step_center(all_predictors()) # %>%
#          prep() %>%
#          bake(new_training)

test_recipe  

new_trained <- test_recipe %>% 
    prep() %>%
    bake(new_training)


new_trained %>%
    group_by(lithology) %>%
    summarize(n = n())


# training_recipe <- recipe(lithology ~ ., data = data_train) %>%
#      step_upsample(lithology, over_ratio = 1, skip = FALSE) %>%
#      step_log(porosity, MS, NRM, RES, chargeability) %>%
#      step_scale(all_predictors()) %>%
#      step_center(all_predictors())
#      #prep()

# training_recipe

# data_trained <- training_recipe %>%
#     prep() %>%
#     bake(data_train) %>%
#     filter(!is.na(lithology))



# training_recipe_2 <- recipe(lithology ~ ., data = data_train) %>%
# #      step_upsample(lithology, over_ratio = 1, skip = FALSE) %>%
#      step_log(porosity, MS, NRM, RES, chargeability) %>%
#      step_scale(all_predictors()) %>%
#      step_center(all_predictors())
#      #prep()

# training_recipe_2 





# data_trained_2 <- training_recipe_2 %>%
#     prep() %>%
#     bake(data_train) %>%
#     filter(!is.na(lithology))

# data_trained
# data_trained_2





lithology,density,porosity,MS,NRM,RES,chargeability
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Igneous,2.590,2.39,-5.60e-06,0.000313,31900,8.304
Metamorphic,2.665,0.35,-1.60e-06,0.000587,3540,1.535
Sedimentary,2.636,10.66,-1.51e-06,0.000392,964,0.299
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Metamorphic,2.634,1.24,-5.64e-06,0.000193,7320,5.230
Metamorphic,2.654,0.30,-2.10e-06,0.002510,3470,2.906
Metamorphic,2.644,0.67,-4.34e-06,0.000255,13200,2.330


Data Recipe

Inputs:

      role #variables
   outcome          1
 predictor          6

Operations:

Log transformation on porosity, MS, NRM, RES, chargeability
Scaling for all_predictors()
Centering for all_predictors()

`summarise()` ungrouping output (override with `.groups` argument)



lithology,n
<fct>,<int>
Igneous,212
Metamorphic,555
Sedimentary,125


## Hi Naijide: try vfold on data_trained and data_trained_2, see if upsample really increase the accuracy 
## then change recipe from all_predictors to just the two predictors  
  
  
## I personally as a geologist, I think these two predictors are closely related to rock type, and use two predictor will be easy on the final visuallization 
## but, who knows, if just two_predictors significantly lower the accuracy compares to all_predictors(), then try add a third one (start with )


In [28]:

training_vfold <- vfold_cv(new_training, v = 5, strata = lithology)

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
       set_engine("kknn") %>%
       set_mode("classification")

knn_results <- workflow() %>%
       add_recipe(test_recipe) %>%
       add_model(knn_tune) %>%
       tune_grid(resamples = training_vfold, grid = 10) %>%
       collect_metrics()

knn_results

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
2,accuracy,multiclass,0.8498337,5,0.01771466,Model01
2,roc_auc,hand_till,0.9149028,5,0.01357698,Model01
3,accuracy,multiclass,0.8756136,5,0.01152917,Model02
⋮,⋮,⋮,⋮,⋮,⋮,⋮
13,roc_auc,hand_till,0.9456554,5,0.01418799,Model09
14,accuracy,multiclass,0.8532044,5,0.01415503,Model10
14,roc_auc,hand_till,0.9457860,5,0.01449051,Model10


  
In the next step, we visualized the data in a scatter plot of resistivity vs porosity as predictors and colored the rock categories.  


### New from Yun: 

#### Now, we can proper visualize data without using any log() or scale_log() functions 

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
initial_graph <- data_train %>%
#     ggplot(aes(x = porosity, y = RES, color = lithology)) +
#      scale_x_log10(labels = comma) +
#      scale_y_log10(labels = comma) +
#    ggplot(aes(x = log(porosity), y = log(RES), color = lithology)) +
   ggplot(aes(x = porosity, y = RES, color = lithology)) +
    geom_point() +
    facet_grid(. ~ lithology) +
    labs(title = "The relationship between Resistivity and Porosity under three Lithology type", x = "Porosity [%]", y = "Resistivity [ohm.m]") +
    theme(text = element_text(size = 20), plot.title = element_text(hjust = 0.5))

initial_graph



current_graph <- data_trained %>%
     ggplot(aes(x = porosity, y = RES, color = lithology)) +
#    ggplot(aes(x = log(porosity), y = log(RES), color = lithology)) +
#      scale_x_continuous(limits = c(-10, 10)) + 
#      scale_y_continuous(limits = c(-10, 10)) + 
#      scale_x_log10(labels = comma) +
#      scale_y_log10(labels = comma) +
    geom_point() +
     facet_grid(. ~ lithology) +
    labs(title = "The relationship between Resistivity and Porosity under three Lithology type", x = "Porosity [%]", y = "Resistivity [ohm.m]") +
    theme(text = element_text(size = 20), plot.title = element_text(hjust = 0.5))

current_graph 

### Methodology
From the 84 columns in the raw data, we select and rename the 8 columns we need. They are: sample type, lithology, grain density, dry bulk density, saturated bulk density, porosity, magnetic susceptibility, natural remanent magnetization, electric resistivity and induced polarization chargeability.  
 
Since Sample type and lithology are categorical variables, we mutate their column type into factor. We then filter the data and only keep the ones with “borehole” sample type and exclude the observation with no specific rock type ("Other"). We split the data into training sets and testing sets and perform an initial analysis on the training set.  
  
The group predicted that resistivity is inversely related to porosity, because the fluid that fills in the pores is generally more conductive than rock itself. The group also want to see if porosity and resistivity alone are sufficient to predict the rock type.  
  

### Preliminary Result 
  
We observed that there is a linear relationship because the points on the scatterplot resemble close to a straight line. The two variables have a weak negative association when the lithology is Igneous because the points are more spread across the graph. There is a moderately strong negative association in the graph for Metamorphic and Sedimentary because as the percentage of Porosity increases, the Resistivity decreases.

The primary data analysis confirmed that there is a weak negative linear relationship between the two factors. However, in the primary visualization, the classification of rock types based on only these two factors are not as distinctive as we imagined.   
  
The group will continue the model building with other necessary steps including standardization or add multiple predictors into the training set in the next step of the project development.   
  
If successful, our model can be used to predict rock formation in an underground setting where human accessibility is limited and only accessible by data acquisition equipment.  
  



### References
Enkin, R.J., 2018. Canadian rock physical property database: first public release; Geological Survey of Canada, Open File 8460, 1 .zip file. https://doi.org/10.4095/313389

